In [1]:
%load_ext autoreload
%autoreload complete

In [ ]:
import torch
from TCN import Chomp1d
t = torch.Tensor([[1, 2, 3], [4, 5, 6], [4, 5, 6]])
chomper = Chomp1d(chomp_size=3)
chomper(t)

In [22]:
t = torch.zeros(size=(3, 3, 4))
chomp = t[:-2]
# print(chomp)
chomp.shape

torch.Size([1, 3, 4])

In [25]:
t = torch.Tensor([1, 2, 3, 4])
chomp = t[:-3]
print(chomp)
chomp.shape

tensor([1.])


torch.Size([1])